In [129]:
import pandas as pd
import numpy as np

from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR, SVC

In [130]:
use_descriptors = True
use_fingerprints = False

regression = True

models_with_parameters = [
['rf', {
#  "bootstrap": True,
#  #"criterion": "entropy",
#  #"criterion": "squared_error",
#  "criterion": "entropy",
#  "min_samples_split": 32,
#  "n_estimators": 30
}],
['lr', {
}],
['nn', {
}],
['gb', {
}],
['sv', {
}]
]

metrics = []


### Retrieving data the old way:

In [131]:
### Input standard SMILES column
def CalculateMorganFingerprint(mol):
    mol = mol.apply(Chem.MolFromSmiles)
    mfpgen = AllChem.GetMorganGenerator(radius=2,fpSize=2048)
    fingerprint = np.array([mfpgen.GetFingerprintAsNumPy(x) for x in mol])
    fingerprint = pd.DataFrame(fingerprint, columns = ['mfp'+str(i) for i in range(fingerprint.shape[1])])
    return fingerprint

### Input standard SMILES column
def CalculateDescriptors(mol):
    mol = mol.apply(Chem.MolFromSmiles)
    calc = Calculator(descriptors, ignore_3D=False)
    X_mordred = calc.pandas(mol, nproc=1)
    X_mordred = X_mordred.select_dtypes(['number'])
    #normalize
    X_mordred = (X_mordred-X_mordred.min())/(X_mordred.max()-X_mordred.min())
    #drop columns wth low std
    X_mordred = X_mordred.loc[:,X_mordred.std()>0.01]
    return X_mordred

In [132]:
def Load_downloaded_CSV_BACE(path, regression = False):
    df = pd.read_csv(path)
    df.drop_duplicates('mol')
    df = df.dropna()
    #df.drop(['CID', 'canvasUID'], axis=1, inplace=True)

    if regression:
        df['Target'] = df['pIC50']
        df.drop('Class', axis=1, inplace=True)
        df.drop('pIC50', axis=1, inplace=True)
    else:
        df['Target'] = df['Class']
        df.drop('Class', axis=1, inplace=True)
        df.drop('pIC50', axis=1, inplace=True)

    df = df[['mol', 'Target', 'Model']]

    if use_descriptors:
        new_df = CalculateDescriptors(df['mol'])
    if use_fingerprints:
        new_df = CalculateMorganFingerprint(df['mol'])
        
    new_df['Target'] = df['Target']
    new_df['Model'] = df['Model']

    return new_df

def Split_downloaded_CSV_BACE(df, scaffold=True):
    if not scaffold:
        X = df.drop(['Target', 'Model'], axis=1)
        y = df[['Target']]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.111, random_state=42)
        return X_train, y_train, X_test, y_test, X_valid, y_valid

    X = df.drop(['Target'], axis=1)
    y = df[['Target', 'Model']]

    X_train = X[X['Model'] == 'Train']
    y_train = y[y['Model'] == 'Train']
    X_test = X[X['Model'] == 'Test']
    y_test = y[y['Model'] == 'Test']
    X_valid = X[X['Model'] == 'Valid']
    y_valid = y[y['Model'] == 'Valid']
    
    X_train = X_train.drop('Model', axis=1)
    y_train = y_train.drop('Model', axis=1)
    X_test = X_test.drop('Model', axis=1)
    y_test = y_test.drop('Model', axis=1)
    X_valid = X_valid.drop('Model', axis=1)
    y_valid = y_valid.drop('Model', axis=1)
    
    return X_train, y_train, X_test, y_test, X_valid, y_valid

In [133]:
df_loaded = Load_downloaded_CSV_BACE(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\BACE\bace.csv", regression=regression)

c:\Users\wojci\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|██████████| 1513/1513 [04:16<00:00,  5.90it/s]
C:\Users\wojci\AppData\Local\Temp\ipykernel_3084\898515923.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['Target'] = df['Target']
C:\Users\wojci\AppData\Local\Temp\ipykernel_3084\898515923.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['Model'] =

In [134]:
scaffold_split = True

X_train, y_train, X_test, y_test, X_valid, y_valid = Split_downloaded_CSV_BACE(df_loaded, scaffold=scaffold_split)

In [135]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(203, 1222)
(1265, 1222)
(45, 1222)


### Configurations

In [136]:
def model_builder(model_name, hyperparams):
    if model_name == 'rf':
        if "n_estimators" not in hyperparams.keys():
            hyperparams["n_estimators"] = 100
        if "min_samples_split" not in hyperparams.keys():
            hyperparams["min_samples_split"] = 2
        if "bootstrap" not in hyperparams.keys():
            hyperparams["bootstrap"] = True  

        if regression:
            if "criterion" not in hyperparams.keys():
                hyperparams["criterion"] = "squared_error"
            model = RandomForestRegressor(n_estimators=hyperparams["n_estimators"],
                                    min_samples_split=hyperparams["min_samples_split"],
                                    criterion=hyperparams["criterion"],
                                    bootstrap=hyperparams["bootstrap"])
        else:
            if "criterion" not in hyperparams.keys():
                hyperparams["criterion"] = "gini"
            model = RandomForestClassifier(n_estimators=hyperparams["n_estimators"],
                                    min_samples_split=hyperparams["min_samples_split"], 
                                    criterion=hyperparams["criterion"],
                                    bootstrap=hyperparams["bootstrap"])
            
    if model_name == 'lr':
        if regression:
            model = LinearRegression()
        else:
            if "C" not in hyperparams.keys():
                hyperparams["C"] = 1
            if "penalty" not in hyperparams.keys():
                hyperparams["penalty"] = "l2"
            if "solver" not in hyperparams.keys():
                hyperparams["solver"] = "liblinear"
            model = LogisticRegression(C=hyperparams["C"], penalty=hyperparams["penalty"], solver=hyperparams["solver"])

    if model_name == 'nn':
        if "hidden_layer_sizes" not in hyperparams.keys():
            hyperparams["hidden_layer_sizes"] = (100,)
        if "activation" not in hyperparams.keys():
            hyperparams["activation"] = "relu"
        if "alpha" not in hyperparams.keys():
            hyperparams["alpha"] = 0.0001  
        if "max_iter" not in hyperparams.keys():
            hyperparams["max_iter"] = 500#200
        if regression:
            model = MLPRegressor(hidden_layer_sizes=hyperparams["hidden_layer_sizes"], activation=hyperparams["activation"], 
                                  alpha=hyperparams["alpha"], max_iter=hyperparams["max_iter"])
        else:
            model = MLPClassifier(hidden_layer_sizes=hyperparams["hidden_layer_sizes"], activation=hyperparams["activation"], 
                                  alpha=hyperparams["alpha"], max_iter=hyperparams["max_iter"])
    if model_name == 'gb':
        if "n_estimators" not in hyperparams.keys():
            hyperparams["n_estimators"] = 100
        if "learning_rate" not in hyperparams.keys():
            hyperparams["learning_rate"] = 0.1
        if regression:
            model = GradientBoostingRegressor(n_estimators=hyperparams["n_estimators"], learning_rate=hyperparams["learning_rate"])
        else:
            model = GradientBoostingClassifier(n_estimators=hyperparams["n_estimators"], learning_rate=hyperparams["learning_rate"])

    if model_name == 'sv':
        if "C" not in hyperparams.keys():
            hyperparams["C"] = 1
        if "degree" not in hyperparams.keys():
            hyperparams["degree"] = 3
        if "kernel" not in hyperparams.keys():
            hyperparams["kernel"] = "rbf"
        if regression:
            if "epsilon" not in hyperparams.keys():
                hyperparams["epsilon"] = 0.1
            model = SVR(C=hyperparams["C"], degree=hyperparams["degree"], kernel=hyperparams["kernel"], epsilon=hyperparams["epsilon"])
        else:
            model = SVC(C=hyperparams["C"], degree=hyperparams["degree"], kernel=hyperparams["kernel"])
    return model
    

In [137]:
def train_and_test(model, X_train, y_train, X_test, y_test, X_valid, y_valid, regression, metrics=[], iterations=1):
    for i in range(iterations):
        model.fit(X_train, np.reshape(y_train, (-1, )))
        
        y_test_predicted = model.predict(X_test)
        y_valid_predicted = model.predict(X_valid)

        print("Standard train-test results:")

        if regression:
            if 'rmse' in metrics or len(metrics) == 0:
                metric_test = mean_squared_error(y_test, y_test_predicted, squared=False)
                metric_valid = mean_squared_error(y_valid, y_valid_predicted, squared=False)
                print("rmse:")
                print(metric_test)
                print(metric_valid)
            if 'mse' in metrics or len(metrics) == 0:
                metric_test = mean_squared_error(y_test, y_test_predicted)
                metric_valid = mean_squared_error(y_valid, y_valid_predicted)
                print("mse:")
                print(metric_test)
                print(metric_valid)
            if 'mae' in metrics or len(metrics) == 0:
                metric_test = mean_absolute_error(y_test, y_test_predicted)
                metric_valid = mean_absolute_error(y_valid, y_valid_predicted)
                print("mae:")
                print(metric_test)
                print(metric_valid)
            if 'r2' in metrics or len(metrics) == 0:
                metric_test = r2_score(y_test, y_test_predicted)
                metric_valid = r2_score(y_valid, y_valid_predicted)
                print("r2:")
                print(metric_test)
                print(metric_valid)
            
        else:
            if 'roc_auc' in metrics or len(metrics) == 0:
                metric_test = roc_auc_score(y_test, y_test_predicted)
                metric_valid = roc_auc_score(y_valid, y_valid_predicted)
                print("roc_auc:")
                print(metric_test)
                print(metric_valid)
            if 'accuracy' in metrics or len(metrics) == 0:
                metric_test = accuracy_score(y_test, y_test_predicted)
                metric_valid = accuracy_score(y_valid, y_valid_predicted)
                print("accuracy:")
                print(metric_test)
                print(metric_valid)
            if 'precision' in metrics or len(metrics) == 0:
                metric_test = precision_score(y_test, y_test_predicted)
                metric_valid = precision_score(y_valid, y_valid_predicted)
                print("precision:")
                print(metric_test)
                print(metric_valid)
            if 'recall' in metrics or len(metrics) == 0:
                metric_test = recall_score(y_test, y_test_predicted)
                metric_valid = recall_score(y_valid, y_valid_predicted)
                print("recall:")
                print(metric_test)
                print(metric_valid)
            if 'f1' in metrics or len(metrics) == 0:
                metric_test = f1_score(y_test, y_test_predicted)
                metric_valid = f1_score(y_valid, y_valid_predicted)
                print("f1:")
                print(metric_test)
                print(metric_valid)
                

def benchmark_train_and_test(model, X_train, y_train, X_test, y_test, X_valid, y_valid, regression, metrics=[], iterations=1):
    model = dc.models.SklearnModel(model) ### for benchmark 
    for i in range(iterations):

        train_set = dc.data.DiskDataset.from_numpy(X=X_train.to_numpy(), y=y_train.to_numpy(), w=np.ones_like(y_train), ids=np.array([i for i in range(y_train.shape[0])]), tasks=['Class'])
        test_set = dc.data.DiskDataset.from_numpy(X=X_test.to_numpy(), y=y_test.to_numpy(), w=np.ones_like(y_test), ids=np.array([i for i in range(y_test.shape[0])]), tasks=['Class'])
        valid_set = dc.data.DiskDataset.from_numpy(X=X_valid.to_numpy(), y=y_valid.to_numpy(), w=np.ones_like(y_valid), ids=np.array([i for i in range(y_valid.shape[0])]), tasks=['Class'])

        model.fit(train_set)
        
        used_metrics = []
        if regression:
            if 'rmse' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.rms_score, np.mean))
            if 'mse' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.mean_squared_error, np.mean))
            if 'mae' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.mean_absolute_error, np.mean))
            if 'r2' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.r2_score, np.mean))

        else:
            if 'roc_auc' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean))
            if 'accuracy' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.accuracy_score, np.mean))
            if 'precision' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.precision_score, np.mean))
            if 'recall' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.recall_score, np.mean))
            if 'f1' in metrics or len(metrics) == 0:
                used_metrics.append(dc.metrics.Metric(dc.metrics.f1_score, np.mean))

        print("Train-test results using benchmark methodology:")
        test_metric = model.evaluate(test_set, used_metrics) #, transformers)
        valid_metric = model.evaluate(valid_set, used_metrics) #, transformers)

        print(test_metric)
        print(valid_metric)

In [138]:
for model_name, hyperparams in models_with_parameters:
    print(model_name)
    model = model_builder(model_name, hyperparams)
    if model_name != "nn":
        benchmark_train_and_test(model, X_train, y_train, X_test, y_test, X_valid, y_valid, regression=regression, metrics=metrics)
    train_and_test(model, X_train, y_train, X_test, y_test, X_valid, y_valid, regression=regression, metrics=metrics)
    

rf
Train-test results using benchmark methodology:
{'mean-rms_score': 1.1012612995045392, 'mean-mean_squared_error': 1.2127764497864266, 'mean-mean_absolute_error': 0.8069857359289431, 'mean-r2_score': 0.2543712610111253}
{'mean-rms_score': 1.3976856870525358, 'mean-mean_squared_error': 1.9535252797915192, 'mean-mean_absolute_error': 1.112176113620265, 'mean-r2_score': 0.5020960333757325}
Standard train-test results:
rmse:
1.104199057928028
1.4252357219808685
mse:
1.2192555595291446
2.0312968632103274
mae:
0.8089775387302504
1.1254257780044445
r2:
0.25038783073583915
0.48227403246514944
lr
Train-test results using benchmark methodology:
{'mean-rms_score': 127047180.50147216, 'mean-mean_squared_error': 1.6140986073373646e+16, 'mean-mean_absolute_error': 92728804.97208469, 'mean-r2_score': -9923661606428792.0}
{'mean-rms_score': 173583140.66615406, 'mean-mean_squared_error': 3.0131106723525824e+16, 'mean-mean_absolute_error': 128864218.27741078, 'mean-r2_score': -7679653655684303.0}
Stan